In [4]:
ord("B")

66

In [12]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1 
        return counts 


In [ ]:
top_pair = max(stats, key=stats.get)

In [9]:
def merge(ids, pair, idx):
    # in the list of ints (ids), replace all consective occurences of pait with the new token idx 
    newids = []
    i = 0
    while i < len(ids):
        # if we are at the very last position AND the pair matches, replace it 
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2 
        else:
            newids.append(ids[i])
            i += 1 
    return newids 


print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99)) 


[5, 6, 99, 9, 1]


In [10]:
text = """A programmer's introduction to Unicode March 3, 2017 Coding . 22 Comments Unicode!"""
tokens = text.encode('utf-8') # raw bytes 
tokens = list(map(int, tokens))  # convert to a list of integers in range 0..255 for convenience

In [15]:
#  --- 
vocab_size = 276 # the desired final vocabulary size 
num_merges = vocab_size - 256 
ids = list(tokens)  


merges = {}  # (int, int) -> int 
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i 
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

merging (65, 32) into a new token 256
merging (256, 112) into a new token 257
merging (257, 114) into a new token 258
merging (258, 111) into a new token 259
merging (259, 103) into a new token 260
merging (260, 114) into a new token 261
merging (261, 97) into a new token 262
merging (262, 109) into a new token 263
merging (263, 109) into a new token 264
merging (264, 101) into a new token 265
merging (265, 114) into a new token 266
merging (266, 39) into a new token 267
merging (267, 115) into a new token 268
merging (268, 32) into a new token 269
merging (269, 105) into a new token 270
merging (270, 110) into a new token 271
merging (271, 116) into a new token 272
merging (272, 114) into a new token 273
merging (273, 111) into a new token 274
merging (274, 100) into a new token 275


In [20]:
print("tokens length:", len(tokens))
print("ids length", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 82
ids length 62
compression ratio: 1.32X


In [26]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given ids (list of integets), return Python string 
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")

    return text 

print(decode([128]))

�


In [32]:
merges

{(65, 32): 256,
 (256, 112): 257,
 (257, 114): 258,
 (258, 111): 259,
 (259, 103): 260,
 (260, 114): 261,
 (261, 97): 262,
 (262, 109): 263,
 (263, 109): 264,
 (264, 101): 265,
 (265, 114): 266,
 (266, 39): 267,
 (267, 115): 268,
 (268, 32): 269,
 (269, 105): 270,
 (270, 110): 271,
 (271, 116): 272,
 (272, 114): 273,
 (273, 111): 274,
 (274, 100): 275}

In [33]:
def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) > 2: 
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))

        if pair not in merges:
            break #  nothing else can be merged 
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens 
        


print(encode(""))

[]


In [34]:
print(decode(encode("hello world")))

hello world


In [35]:
text2  = decode(encode(text))
print(text2 == text)

True


In [36]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treate as specific to any given writing systems"
valtext2 = decode(encode(valtext))

print(valtext == valtext)

True


In [ ]:
import regex as re 

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+}""")
print(re.findall(gpt2pat, "Hello world"))

['Hello', ' world']


In [39]:
example = """for  i in range(1, 101):
    if i % 3 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else: 
        print(i)
"""

print(re.findall(gpt2pat, example))

['for', ' ', ' i', ' in', ' range', '(', '1', ',', ' 101', '):', '\n   ', ' if', ' i', ' %', ' 3', ' and', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'FizzBuzz', '")', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Fizz', '")', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Buzz', '")', '\n   ', ' else', ':', ' \n       ', ' print', '(', 'i', ')', '\n']


In [ ]:
import tiktoken 

# GPT-2 
enc = tiktoken.get_encoding("gpt-2")
print(enc.encode("    Hello world!!!"))


# GPT-4 (merge spaces)
enc = tiktoken.get_encoding("cli100k_base")
print(enc.encode("   hello world!!!"))


In [40]:
import re 
from collections import Counter, defaultdict 

In [45]:
class SimpleBPETokenizer: 
    def __init__(self):
        # initial vocabulary with common characters an symbols 
        self.vocab = {chr(i): i for i in range(32, 127)} # ASCII printable chars 
        self.merges = {}
        self.word_to_tokens = {}

    def get_stats(self, word_freq): 
        """"Count pairs of adjacent symbols in with word freqeuncy dictionary. """
        pairs = Counter()
        for word, freq in word_freq.items():
            symbols = word.split()
            for i in range(len(symbols) - 1): 
                pairs[(symbols[i], symbols[i + 1])] += freq
        return pairs 

    def merge_vocab(self, pair, word_freq):
        """Merge the most frequent pair in the vocabulary."""
        new_word_freq = {}
        pattern = re.escape(' '.join(pair))
        replacement = ''.join(pair)

        for word in word_freq:
            new_word = re.sub(pattern, replacement, word)
            new_word_freq[new_word] = word_freq[word]
            return new_word_freq
    
    def train(self, text, num_merges=100):
        """Train the tokenizer on text with a specified number of merges."""
        # Preprocess text: split into words and add spaces between characters 
        words = text.lower().split()
        word_freq = Counter(words)

        # Convert words to characters sequences with spaces 
        char_words = {}
        for word, freq in word_freq.items():
            char_word = ' '.join(list(word)) + ' </w>'
            char_words[char_word] = freq 

            # Perform BPE merges ]
            for i in range(num_merges):
                pairs = self.get_stats(char_words)
                if not pairs:
                    break 
                best_pair = pairs.most_common(1)[0][0]
                self.merges[best_pair] = i 
                char_words = self.merge_vocab(best_pair, char_words)
            
            # Build final vocabulary
            self.vocab.upate({''.join(pair): len(self.vocab) + i for pair, i in self.merges.items()}) 
            self.word_to_tokens = char_words 

    def tokenize(self, text):
        """Tokenize new text based on trained vocabulary"""
        if not self.merges:
            raise ValueError("Tokenizer must be trained first!")
    
        # Preprocess input text 
        words = text.lower().split()
        tokens = []

        for word in words:
            # start with characters separated by spaces 
            current = ' '.join(list(word)) + ' </w>'

            # Apply all learned merges 
            while True:
                pairs = [(current.split()[i], current.split()[i +1]) for i in range(len(current.split()) -1 )]
                valid_pairs = [p for p in pairs if p in self.merges] 
                if not valid_pairs:
                    break

                # Find teh earliest merge 
                best_pair = min(valid_pairs, key=lambda x: self.merges[x])
                pattern = re.escape(' '.join(best_pair))
                replacement = ''.join(best_pair)
                current = re.sub(pattern, replacement, current)

                # convert to token IDs 
                token_words = current.split()
                token_ids = [self.vocab.get(token, self.vocab['<unk>']) for token in token_words if token != '</w>']
                tokens.extend(token_ids)

            return tokens 



{'\x00': 0,
 '\x01': 1,
 '\x02': 2,
 '\x03': 3,
 '\x04': 4,
 '\x05': 5,
 '\x06': 6,
 '\x07': 7,
 '\x08': 8,
 '\t': 9,
 '\n': 10,
 '\x0b': 11,
 '\x0c': 12,
 '\r': 13,
 '\x0e': 14,
 '\x0f': 15,
 '\x10': 16,
 '\x11': 17,
 '\x12': 18,
 '\x13': 19,
 '\x14': 20,
 '\x15': 21,
 '\x16': 22,
 '\x17': 23,
 '\x18': 24,
 '\x19': 25,
 '\x1a': 26,
 '\x1b': 27,
 '\x1c': 28,
 '\x1d': 29,
 '\x1e': 30,
 '\x1f': 31,
 ' ': 32,
 '!': 33,
 '"': 34,
 '#': 35,
 '$': 36,
 '%': 37,
 '&': 38,
 "'": 39,
 '(': 40,
 ')': 41,
 '*': 42,
 '+': 43,
 ',': 44,
 '-': 45,
 '.': 46,
 '/': 47,
 '0': 48,
 '1': 49,
 '2': 50,
 '3': 51,
 '4': 52,
 '5': 53,
 '6': 54,
 '7': 55,
 '8': 56,
 '9': 57,
 ':': 58,
 ';': 59,
 '<': 60,
 '=': 61,
 '>': 62,
 '?': 63,
 '@': 64,
 'A': 65,
 'B': 66,
 'C': 67,
 'D': 68,
 'E': 69,
 'F': 70,
 'G': 71,
 'H': 72,
 'I': 73,
 'J': 74,
 'K': 75,
 'L': 76,
 'M': 77,
 'N': 78,
 'O': 79,
 'P': 80,
 'Q': 81,
 'R': 82,
 'S': 83,
 'T': 84,
 'U': 85,
 'V': 86,
 'W': 87,
 'X': 88,
 'Y': 89,
 'Z': 90,
 '[': 91,
